- Helper to AutomotiveInteraction.jl
- Perform non-linear least squares fit using Levenberg-Marquardt
- Adapted from Jeremy Morton's code in LSTM-acc-predict
- The IDM features and true acceleration value is provided by the Julia code in the form of .txt file
- Should think about integrating python part i.e. this file directly using PyCall but leave for later

In [1]:
from lmfit import minimize, Parameters, Parameter, report_fit
import math
import numpy as np

In [2]:
# Function to minimize
def IDM(params, i, true_acc):
    dmn = params['dmn'].value
    T = params['T'].value
    bcmf = params['bcmf'].value
    smx = params['smx'].value
    amx = params['amx'].value
    
    d = x[i, 0]
    r = x[i, 1]
    s = x[i, 2]
    
    d_des = dmn + T*s - s*r/2/math.sqrt(amx*bcmf)
    pred = amx*(1 - (s/smx)**4 - (d_des/d)**2)
    
    return pred - true_acc

In [3]:
# Define parameters; outcome is not particularly sensitive to dmn or smx values so boundaries
# are (admittedly) a little arbitrary
params = Parameters()
params.add('dmn', value = 6., min = 0.1, max = 5.25) 
params.add('T', value = 1., min = 0.1, max = 5.0)
params.add('bcmf', value = 1.0, min = 0.5, max = 5.0) 
params.add('smx', value = 15.0, min = 1.0, max = 30.0)
params.add('amx', value = 1.0, min = 0.75, max = 3.0)

In [ ]:
result = minimize(IDM, params, args=(range(len(acc)), acc))

In [ ]:
result.params

In [ ]:
sum(abs(result.residual))/count